In [19]:
!pip install --upgrade pip
!pip install langchain_huggingface sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 121.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 77.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 85.5 MB/s  0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=a1b2ac9ef7628fb61e93c61a478c5e

In [111]:
import os
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

def file_save(file,data):
    with open(file, 'w', encoding="utf-8") as f:
        json.dump(data, f)
def load_file(file):
  with open(file, 'r', encoding="utf-8") as f:
    json_to_dic = json.load(f)
  return json_to_dic

In [43]:
import os
from getpass import getpass

if not os.getenv("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")

Enter your Mistral API Key: ··········


In [112]:
llm = ChatMistralAI(model="mistral-tiny")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

prompt = ChatPromptTemplate.from_template("""
      Answer the following question based only on the provided context:

      <context>
      {context}
      </context>

      Question: {input}
      """)

# data.update(new_data)
# file_save(file,data)

In [100]:
from langchain_text_splitters import PythonCodeTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from google.colab import files
# uploaded = files.upload()
pdf = PyPDFLoader("/content/Day-9.pdf")
data = pdf.load()

file = "chunking_analysis.json"
chunk_analyse={
    "chunk_size":[],
    "chunk_overlap":[],
    "chunk_count":[],
    "what is embeddings ?":[]
}

print(len(data))

for chunk_size in range(50,1001,50):
  for lap_size in range(50,200,20):

    if chunk_size<lap_size:
      continue

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=lap_size)
    chunks = splitter.split_documents(data)

    chunk_analyse["chunk_size"].append(chunk_size)
    chunk_analyse["chunk_overlap"].append(lap_size)
    chunk_analyse["chunk_count"].append(len(chunks))

    vectorstore = FAISS.from_documents(chunks, embeddings)

    retriever = vectorstore.as_retriever(search_kwargs={"k": 1})



    doc_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

    rag = create_retrieval_chain(retriever,doc_chain)

    input_user = "what is embeddings ?"

    response = rag.invoke({"input":input_user})


    chunk_analyse[input].append(response["answer"])

file_save(file,chunk_analyse)






13


In [101]:
import pandas as pd
df = pd.DataFrame(chunk_analyse)
df.to_csv("chunk_analyse.csv")

In [97]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(data)

c=[chunk.page_content for chunk in chunks]

c_len=[]
for chunk in chunks:
  # print(chunk.page_content)
  c_len.append(len(chunk.page_content))

print("Total chunk size: ",len(c))

print("page no:",end="\t")
for i,v in enumerate(c_len):
  print(f"{i+1}",end="\t")

print()

print("page content:",end="\t")
for v in c_len:
  print(f"{v}",end="\t")


Total chunk size:  41
page no:	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	
page content:	110	274	185	261	297	288	127	258	245	177	241	296	238	84	284	297	279	162	290	266	232	256	14	189	297	257	286	99	286	247	284	207	254	263	164	181	272	294	186	188	227	

In [107]:
print(c[2])
print(len(c[2]))

'numerical representations enable machines to understand and \nprocess human language with unprecedented accuracy, \ntransforming applications from semantic search to advanced AI \nsystems.'